In [18]:
import numpy as np
import xtrack as xt
import xcoll as xc
import xobjects as xo
import xpart as xp

In [19]:
line = xt.Line.from_json('injection_thin_approx_ap2.json')
#line = xt.Line.from_json('lhc_run3_b1.json')

Loading line from dict:   0%|          | 0/31427 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
num_turns = 3000
num_particles = 500

nemitt_x = 3.5e-6
nemitt_y = 3.5e-6
amplitude_adt = 0.12

In [ ]:
print('Install collimator')
coll = xc.EverestCollimator(length=1.83, gap=5, material=xc.materials.Carbon) # length is 1.83
line.collimators.install('tcsm.51932', coll)

#Make aperture for collimator and update line
coll_ap = xt.LimitRectEllipse(a=0.05, b=0.05, max_x=0.05, max_y=0.05) 
coll_ap_names = ['tcsm.51932_aper_upstream', 'tcsm.51932_aper_downstream']
coll_ap_idx = [line.element_names.index('tcsm.51932'), line.element_names.index('tcsm.51932') + 1]

max_length = max(max(map(len, line.element_names)), max(map(len, coll_ap_names)))
element_names = np.array(line.element_names, dtype=f'<U{max_length}')
names = np.array(coll_ap_names, dtype=f'<U{max_length}')
element_names = np.insert(element_names, coll_ap_idx, coll_ap_names)

insert_colls = {name: coll_ap for name in coll_ap_names}

line.element_names = element_names.tolist()
line.element_dict = {**line.element_dict, **insert_colls}

Install collimator
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


Slicing line:   0%|          | 0/30277 [00:00<?, ?it/s]

In [ ]:
adt = xc.BlowUp.install(line, name=f'adt_H_blowup', at_s=line.get_s_position('adkcv.32171'), plane='H', stop_at_turn=num_turns,
                        amplitude=amplitude_adt, use_individual_kicks=True)

In [ ]:
df_with_coll = line.check_aperture()

Checking aperture:   0%|          | 0/30280 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


In [ ]:
# Start interaction record
impacts = xc.InteractionRecord.start(line=line, record_impacts=True)

In [ ]:
line.build_tracker()
tw = line.twiss()
line.collimators.assign_optics(twiss=tw, nemitt_x=nemitt_x, nemitt_y=nemitt_y)
adt.calibrate_by_emittance(nemitt=nemitt_x, twiss=tw)
line.optimize_for_tracking()

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Disable xdeps expressions
Replance slices with equivalent elements
Remove markers
Remove inactive multipoles
Merge consecutive multipoles
Remove redundant apertures
Remove zero length drifts
Merge consecutive drifts
Use simple bends
Use simple quadrupoles
Rebuild tracker data
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [ ]:
line.scattering.disable()
part = xp.generate_matched_gaussian_bunch(num_particles=num_particles, total_intensity_particles=2.2e11,
                                          nemitt_x=3.5e-6, nemitt_y=3.5e-6, sigma_z=0.224, line=line) #Flat bottom: 0.224, flat top: 0.124

*** Maximum RMS bunch length 0.23305718962575075m.
... distance to target bunch length: -2.2400e-01
... distance to target bunch length: 4.2397e-03
... distance to target bunch length: 4.0509e-03
... distance to target bunch length: -5.4163e-03
... distance to target bunch length: 1.6934e-03
... distance to target bunch length: -3.8711e-04
... distance to target bunch length: 6.2540e-05
... distance to target bunch length: 1.9666e-06
... distance to target bunch length: -3.5749e-10
... distance to target bunch length: 9.4807e-08
--> Bunch length: 0.22399999964250966
--> Emittance: 0.34322710188738337


In [20]:
final_res = {}
prev_res = {}
dead_idx = []

line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads=26))

line.scattering.enable()
adt.activate()
for turn in range(num_turns):
    if turn%100==0:
        print(f'Turn {turn}')
    line.track(part, turn_by_turn_monitor='ONE_TURN_EBE')
    res = line.record_last_track
    ids = part.particle_id[part.state<=0]
    if ids.size>0:
        for idx in ids:
            if idx not in dead_idx:
                dead_idx.append(idx)
                if turn not in final_res:
                    final_res[turn] = {}
                #this_idx = res.particle_id.index(idx)
                this_idx = int(np.where(np.any(res.particle_id==idx, axis=1))[0][0])
                final_res[turn][idx] = {
                    'x': res.x[this_idx].copy(),
                    'y': res.y[this_idx].copy()
                }
                if turn > 0:
                    if turn-1 not in final_res:
                        final_res[turn-1] = {}
                    prev_idx = int(np.where(np.any(prev_res['pid']==idx, axis=1))[0][0])
                    final_res[turn-1][idx] = {
                        'x': prev_res['x'][prev_idx].copy(),
                        'y': prev_res['y'][prev_idx].copy()
                    }
    prev_res = {
        'x': res.x.copy(),
        'y': res.y.copy(),
        'pid': res.particle_id.copy()
    }
    del res
adt.deactivate()
line.scattering.disable()

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Turn 0
Turn 100


KeyboardInterrupt: 

In [ ]:
line.track(part, turn_by_turn_monitor='ONE_TURN_EBE')
res = line.record_last_track

In [ ]:
len(res.x[0])

21604

In [ ]:
import pickle

In [ ]:
with open('final_res.pkl', 'wb') as f:
    pickle.dump(final_res, f)

In [ ]:
len(line.get_table().s)

21604

In [ ]:
for turn in final_res:
    if len(final_res[turn]) > 0:
        for id in final_res[turn]:
            if len(final_res[turn][id]['x']) != len(line.get_table().s):
                print(f'Turn {turn}, id {id}, len x: {len(final_res[turn][id]["x"])}')

In [ ]:
final_res['s'] = line.get_table().s

In [ ]:
with open('final_res.pkl', 'rb') as f:
    loaded = pickle.load(f)

In [ ]:
res

NameError: name 'res' is not defined